# SynthRad Demo: Synthetic Radiology Report Generation

This notebook demonstrates how to use the `synthrad` package to generate synthetic lung cancer CT reports.

## Prerequisites

Before running this notebook, ensure you have the required dependencies:

```bash
# From project root:
pip install -e .
pip install -r notebooks/requirements.txt
```

In [4]:
# Import libraries
import sys, os
from pathlib import Path

# Add project source to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Import synthrad
from synthrad.generator import generate_case, generate_report
from synthrad.generator import generate_patient_timeline, write_case

print('✅ SynthRad imported successfully!')

✅ SynthRad imported successfully!


## 1. Basic Case Generation

Let's start by generating a single synthetic lung cancer case.

In [6]:
# Generate a case
case = generate_case(
    seed=42,
    stage_dist={"I": 0.3, "II": 0.3, "III": 0.3, "IV": 0.1},
    patient_id="DEMO01"
)

print(f"Patient: {case.meta.patient_id}")
print(f"Visit Number: {case.meta.visit_number}")
print(f"Accession: {case.meta.accession_number}")
if case.primary:
    print(f"Primary tumor: {case.primary.lobe} lobe, {case.primary.size_mm}mm")
    print(f"Features: {case.primary.features}")
print(f"TNM: {case.tnm.T}/{case.tnm.N}/{case.tnm.M} → Stage {case.tnm.stage_group}")
print(f"Rationale: {case.rationale}")

Patient: DEMO01
Visit Number: 1
Accession: 20201209356787
Primary tumor: RML lobe, 72mm
Features: ['spiculation']
TNM: T3/N2/M0 → Stage III
Rationale: ['T3 because >70 mm', 'N1: station 11L short-axis ≥10 mm (14 mm)', 'N2: station 4L short-axis ≥10 mm (15 mm)', 'N1: station 10R short-axis ≥10 mm (12 mm)', 'No definite distant metastases identified']


In [7]:
# Generate the radiology report
report = generate_report(case)
print("=== Generated Radiology Report ===")
print(report)

=== Generated Radiology Report ===
FINDINGS:
Motion artifact present.
Lungs:
  72 mm right middle lobe mass with irregular, spiculated contour spiculated margins.
Lymph nodes:
  Subcentimeter 11L node measuring 14 mm short-axis.
  Subcentimeter 4L node measuring 15 mm short-axis.
  Subcentimeter 10R node measuring 12 mm short-axis.
Pleura/Pleural spaces: No pleural effusion. 
Mediastinum: Mediastinum normal.
Great vessels/Aorta: Great vessels normal.
Upper abdomen (limited): Abdomen unremarkable.
Bones/Osseous structures: Bones unremarkable.
Metastatic survey: No definite distant metastases identified.

IMPRESSION:
- Primary lung neoplasm in the right middle lobe measuring approximately 72 mm.
- Nodal disease as detailed above.
- No definite distant metastases identified.


## 2. Advanced Parameters

Explore how different parameters affect case generation.

In [6]:
# Generate cases with specific stage distributions
stage_i_cases = []
for seed in range(100, 103):
    case = generate_case(seed=seed, stage_dist={"I": 1.0}, patient_id=f"STAGEI_{seed-100:02d}")
    stage_i_cases.append(case)

print("=== Stage I Cases ===")
for case in stage_i_cases:
    if case.primary:
        print(f"Patient {case.meta.patient_id}: {case.tnm.stage_group} stage, {case.primary.size_mm}mm tumor")

=== Stage I Cases ===
Patient STAGEI_00: I stage, 19mm tumor
Patient STAGEI_01: I stage, 26mm tumor
Patient STAGEI_02: II stage, 36mm tumor


## 3. Patient Timelines

Generate follow-up studies for a patient over time.

In [8]:
# Generate a complete patient timeline
cases, dates = generate_patient_timeline(
    patient_id="P001",
    seed=500,
    stage_dist={"I": 0.2, "II": 0.3, "III": 0.3, "IV": 0.2},
    max_studies=4,
    response_dist={"CR": 0.1, "PR": 0.3, "SD": 0.4, "PD": 0.2}
)

print("=== Patient Timeline (P001) ===")
for i, (case, date) in enumerate(zip(cases, dates)):
    date_str = date.strftime("%Y-%m-%d")
    visit_num = case.meta.visit_number
    stage = case.tnm.stage_group
    print(f"Study {visit_num} ({date_str}): Stage {stage}")
    if case.primary:
        print(f"  Primary: {case.primary.lobe} lobe, {case.primary.size_mm}mm")
    if case.nodes:
        print(f"  Nodes: {len(case.nodes)} enlarged nodes")
    if case.mets:
        print(f"  Metastases: {len(case.mets)} sites")
    print()

=== Patient Timeline (P001) ===
Study 1 (2023-12-05): Stage III
  Primary: LUL lobe, 27mm
  Nodes: 1 enlarged nodes

Study 2 (2024-05-02): Stage I
  Primary: LUL lobe, 10mm

Study 3 (2024-09-05): Stage I
  Primary: LUL lobe, 6mm

Study 4 (2024-11-01): Stage I
  Primary: LUL lobe, 5mm

Study 5 (2025-04-26): Stage I
  Primary: LUL lobe, 5mm



## 4. Data Analysis

Convert cases to JSON format for analysis.

In [11]:
# Convert timeline to JSON
from synthrad.generator import case_to_recist_jsonl
import json

jsonl_data = case_to_recist_jsonl(cases, dates)
print(f"Generated {len(jsonl_data)} study entries")
print()
print("=== First Entry ===")
print(json.dumps(jsonl_data[0], indent=2))

Generated 5 study entries

=== First Entry ===
{
  "patient_id": "P001",
  "timepoint": 0,
  "study_date": "2023-12-05",
  "baseline_sld_mm": 45,
  "current_sld_mm": null,
  "nadir_sld_mm": null,
  "overall_response": "SD",
  "lesions": [
    {
      "lesion_id": "primary_LUL",
      "kind": "primary",
      "organ": "lung",
      "location": "LUL",
      "rule": "longest",
      "baseline_mm": 27,
      "follow_mm": null,
      "size_mm_current": 27,
      "margin": "spiculated",
      "enhancement": "enhancing",
      "necrosis": false,
      "suspicious": true,
      "target": true
    },
    {
      "lesion_id": "ln_2L_0",
      "kind": "ln",
      "organ": "lymph_node",
      "station": "2L",
      "rule": "short_axis",
      "baseline_mm": 18,
      "follow_mm": null,
      "size_mm_current": 18,
      "margin": "smooth",
      "enhancement": "enhancing",
      "necrosis": false,
      "suspicious": true,
      "target": true
    }
  ]
}


## 5. File Output

Save cases to files with text reports and JSON sidecars.

In [12]:
# Save a case to files
output_dir = "demo_output"
os.makedirs(output_dir, exist_ok=True)

# Write case
write_case(case=case, outdir=output_dir, stem="example_case", use_radlex=False)

print("Case saved to files:")
if os.path.exists(output_dir):
    for file in os.listdir(output_dir):
        print(f"  📄 {file}")

Case saved to files:
  📄 P001


## Summary

This notebook demonstrated how to:
- Generate synthetic radiology reports
- Use different parameters for customization
- Create patient timelines with follow-up studies
- Extract structured data for analysis
- Save cases to files

For more examples, see the generated docs and README.md.